In [27]:
import pandas as pd
import math
import snappy
import sage

In [2]:
pd.set_option('display.max_colwidth', None)

### Table of Knot Invariants for $T(4,n)$
Knot signature, Alexander polynomial, Arf, upsilon, etc.

In [25]:
alex = snappy.Link('T(4,5)').alexander_polynomial(norm=False)
alex

t^-12 - t^-11 + t^-8 - t^-6 + t^-4 - t^-1 + 1

In [21]:
type(alex)

sage.rings.polynomial.laurent_polynomial.LaurentPolynomial_univariate

In [32]:
min(alex.exponents())

-12

In [34]:
alex.shift(-min(alex.exponents()) // 2)

t^-6 - t^-5 + t^-2 - 1 + t^2 - t^5 + t^6

In [ ]:
invariants = []
for n in range(5, 20):
    if math.gcd(4,n) != 1: continue

    knot_name = f'T(4,{n})'
    knot = snappy.Link(knot_name)
    sage_knot = knot.sage_link()

    floer = knot.knot_floer_homology()
    #print(floer)

    alexander = knot.alexander_polynomial(norm=False)

    # get symmetric presentation of alexander polynomial
    alexander_sym = alexander.shift(-min(alex.exponents()) // 2) 
    
    invariants.append([n, knot_name, knot.signature(), sage_knot.arf_invariant(), floer['nu'], alexander_sym])
    
df = pd.DataFrame(data=invariants, columns=['n', 'Knot', 'Signature', 'Arf', 'floer nu', 'Alexander'])
df

Observe that Remark 2.2 from Fairchild, et al. holds:

For odd $q$,
$$
\text{Arf}(T(p,q)) =
\begin{cases}
0 & p \text{ odd or } q\equiv\pm1\bmod 8, \\
1 & p \text { even and } q \equiv\pm 3\bmod 8.
\end{cases}
$$

Let's see for which $n$ does $\gamma_4(T(4,n))\ge 2$ using Proposition 2.3 from Fairchild, et al.

Proposition 2.3. Given a knot $K$ in $S^3$, if $\sigma(K)+4\text{Arf}(K)\equiv 4\bmod 8$ then $\gamma_4(K)\ge2$.

In [4]:
combined = df['Signature'] + 4 * df['Arf']
combined_mod = combined % 8
pd.DataFrame(data=zip(df['n'], combined, combined_mod),
             columns=['n', '$\\sigma(K)+4\\text{Arf}(K)$', '$\\sigma(K)+4\\text{Arf}(K)\\bmod8$'])

,n,$\sigma(K)+4\text{Arf}(K)$,$\sigma(K)+4\text{Arf}(K)\bmod8$
0,5,-4,4
1,7,-14,2
2,9,-16,0
3,11,-18,6
4,13,-20,4
5,15,-30,2
6,17,-32,0
7,19,-34,6


Now let's compute the lower bound in Theorem 2.6, again from Fairchild, et al.

For a knot $K$ in $S^3$,
$$
\left| \nu(K)+\frac{\sigma(K)}{2} \right| \le \gamma_4(K).
$$

In [5]:
lb = abs(df['floer nu'] + 0.5 * df['Signature'])

In [6]:
pd.DataFrame(data=zip(df['n'], lb), columns=['n', '$\\left| \\nu(K)+\\frac{\\sigma(K)}{2} \\right|$']).to_latex()

ImportError: Pandas requires version '3.1.5' or newer of 'jinja2' (version '3.1.4' currently installed).

In [ ]:
from sympy import latex
col_names = ['n', '\\left| \\nu(K)+\\frac{\\sigma(K)}{2} \\right|']
col_names = [n
latex_col_names = map(lambda tc: '$'+latex(tc)+'$', col_names)


In [ ]:
list(latex_col_names)